In [227]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import wikipedia
import json
import xml.etree.ElementTree as ET
import xml.dom.minidom 

In [232]:
print("¿Cual es el identificador de la clase de la cual se quiere generar un diagrama?")
clase = input()

¿Cual es el identificador de la clase de la cual se quiere generar un diagrama?
Q68


In [233]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url)    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [234]:
#Aquí se genera la consulta. La primera es para obtener el numero de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [235]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q278299,literal,Q278299,NaN
1,uri,http://www.wikidata.org/entity/Q392576,literal,PERM,es
2,uri,http://www.wikidata.org/entity/Q770597,literal,Colossus,es
3,uri,http://www.wikidata.org/entity/Q1151478,literal,Q1151478,NaN
4,uri,http://www.wikidata.org/entity/Q1151495,literal,Q1151495,NaN


In [236]:
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

144

In [237]:
#Aquí se genera la consulta. Esta se hace para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [238]:
#Se confirma que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P276,literal,http://www.wikidata.org/prop/direct/P276
2,uri,http://www.wikidata.org/prop/direct/P277,literal,http://www.wikidata.org/prop/direct/P277
3,uri,http://www.wikidata.org/prop/direct/P287,literal,http://www.wikidata.org/prop/direct/P287
4,uri,http://www.wikidata.org/prop/direct/P306,literal,http://www.wikidata.org/prop/direct/P306


In [239]:
#Se listan las propiedades de salida de la clase.
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)
lista_propiedades

['P279',
 'P276',
 'P277',
 'P287',
 'P306',
 'P361',
 'P373',
 'P571',
 'P580',
 'P585',
 'P625',
 'P646',
 'P724',
 'P729',
 'P856',
 'P910',
 'P935',
 'P941',
 'P973',
 'P1296',
 'P1324',
 'P1417',
 'P1448',
 'P1813',
 'P1889',
 'P2013',
 'P2581',
 'P2669',
 'P3222',
 'P3219',
 'P3417',
 'P4342',
 'P5019',
 'P5936',
 'P6366',
 'P7818',
 'P7982',
 'P8349',
 'P9100',
 'P18',
 'P17',
 'P31',
 'P138',
 'P159',
 'P156',
 'P154',
 'P170',
 'P178',
 'P176',
 'P227',
 'P244',
 'P495',
 'P576',
 'P577',
 'P880',
 'P1036',
 'P1366',
 'P1365',
 'P1430',
 'P2043',
 'P2048',
 'P2049',
 'P2671',
 'P2664',
 'P3985',
 'P7772',
 'P127',
 'P136',
 'P144',
 'P155',
 'P162',
 'P186',
 'P195',
 'P217',
 'P527',
 'P582',
 'P1065',
 'P1092',
 'P1482',
 'P1535',
 'P1552',
 'P1705',
 'P1814',
 'P2046',
 'P2283',
 'P2791',
 'P3342',
 'P8408',
 'P61',
 'P131',
 'P366',
 'P641',
 'P1716',
 'P2031',
 'P2130',
 'P2209',
 'P2560',
 'P5204',
 'P7704',
 'P137',
 'P1436',
 'P2149',
 'P2284',
 'P2928',
 'P5868']

In [291]:
for prop in lista_propiedades:
    #Se obtienen de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
    #Aquí se genera la consulta. Esta se hace para obtener las propiedades de la clase.
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    es_OP = string[105:109]
    print(es_OP)
    if es_OP == "true":
        #Aquí generamos la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad. Se realiza esta
        # consulta para todas las object properties encontradas anteriormente.
        sparql_query4 = """SELECT DISTINCT ?claseInicial ?claseInicialLabel ?prop ?propLabel ?Clase ?ClaseLabel
        WHERE {
        #occupation (P106) must be Formula 1 driver (Q10841764)
        ?claseInicial wdt:P31 wd:"""+clase+""" .
        ?claseInicial wdt:"""+prop +""" ?prop .
        ?prop wdt:P31 ?Clase .
        #to obtain labels in spanish:
        SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
        }  
        """
        #para consultar otro endpoint, cambiar la URL del servicio y la consulta.
        sparql_service_url = "https://query.wikidata.org/sparql"
        clases_object_property_wikidata = query_wikidata(sparql_query4, sparql_service_url)
        print(clases_object_property_wikidata)

true
Empty DataFrame
Columns: []
Index: []
true
   Clase.type                               Clase.value ClaseLabel.type  \
0         uri      http://www.wikidata.org/entity/Q3914         literal   
1         uri   http://www.wikidata.org/entity/Q1187811         literal   
2         uri   http://www.wikidata.org/entity/Q1093436         literal   
3         uri   http://www.wikidata.org/entity/Q2398990         literal   
4         uri    http://www.wikidata.org/entity/Q134626         literal   
5         uri    http://www.wikidata.org/entity/Q200250         literal   
6         uri   http://www.wikidata.org/entity/Q1066984         literal   
7         uri   http://www.wikidata.org/entity/Q1180262         literal   
8         uri   http://www.wikidata.org/entity/Q1549591         literal   
9         uri     http://www.wikidata.org/entity/Q38723         literal   
10        uri      http://www.wikidata.org/entity/Q3918         literal   
11        uri   http://www.wikidata.org/entity/Q1637

true
  Clase.type                               Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q21562092         literal   
1        uri      http://www.wikidata.org/entity/Q9143         literal   
2        uri    http://www.wikidata.org/entity/Q899523         literal   
3        uri      http://www.wikidata.org/entity/Q9143         literal   
4        uri   http://www.wikidata.org/entity/Q7998546         literal   
5        uri  http://www.wikidata.org/entity/Q12772052         literal   

                               ClaseLabel.value ClaseLabel.xml:lang  \
0           lenguaje de programación imperativo                  es   
1                      lenguaje de programacion                  es   
2  lenguaje de programación orientado a objetos                  es   
3                      lenguaje de programacion                  es   
4                                      Q7998546                 NaN   
5       lenguaje de programación multiparadigma   

true
   Clase.type                               Clase.value ClaseLabel.type  \
0         uri      http://www.wikidata.org/entity/Q9135         literal   
1         uri  http://www.wikidata.org/entity/Q15545584         literal   
2         uri    http://www.wikidata.org/entity/Q218616         literal   
3         uri      http://www.wikidata.org/entity/Q9135         literal   
4         uri      http://www.wikidata.org/entity/Q9135         literal   
5         uri     http://www.wikidata.org/entity/Q14656         literal   
6         uri  http://www.wikidata.org/entity/Q28933706         literal   
7         uri     http://www.wikidata.org/entity/Q14656         literal   
8         uri     http://www.wikidata.org/entity/Q11368         literal   
9         uri      http://www.wikidata.org/entity/Q9135         literal   
10        uri      http://www.wikidata.org/entity/Q9135         literal   
11        uri      http://www.wikidata.org/entity/Q7397         literal   
12        uri     ht

  Clase.type                               Clase.value ClaseLabel.type  \
0        uri   http://www.wikidata.org/entity/Q2385804         literal   
1        uri    http://www.wikidata.org/entity/Q957055         literal   
2        uri  http://www.wikidata.org/entity/Q20937557         literal   

        ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0  institución educativa                  es               uri   
1                Q957055                 NaN               uri   
2                  serie                  es               uri   

                        claseInicial.value claseInicialLabel.type  \
0   http://www.wikidata.org/entity/Q169399                literal   
1   http://www.wikidata.org/entity/Q138875                literal   
2  http://www.wikidata.org/entity/Q5010338                literal   

    claseInicialLabel.value claseInicialLabel.xml:lang prop.type  \
0                     ENIAC                         es       uri   
1  Apollo Guidance Comput

true
    Clase.type                               Clase.value ClaseLabel.type  \
0          uri    http://www.wikidata.org/entity/Q151885         literal   
1          uri     http://www.wikidata.org/entity/Q39546         literal   
2          uri    http://www.wikidata.org/entity/Q151885         literal   
3          uri     http://www.wikidata.org/entity/Q39546         literal   
4          uri    http://www.wikidata.org/entity/Q151885         literal   
5          uri    http://www.wikidata.org/entity/Q151885         literal   
6          uri     http://www.wikidata.org/entity/Q39546         literal   
7          uri     http://www.wikidata.org/entity/Q39546         literal   
8          uri     http://www.wikidata.org/entity/Q39546         literal   
9          uri     http://www.wikidata.org/entity/Q39546         literal   
10         uri     http://www.wikidata.org/entity/Q39546         literal   
11         uri     http://www.wikidata.org/entity/Q39546         literal   
12     

...

fals
fals
true
  Clase.type                               Clase.value ClaseLabel.type  \
0        uri      http://www.wikidata.org/entity/Q5300         literal   
1        uri  http://www.wikidata.org/entity/Q10929058         literal   
2        uri  http://www.wikidata.org/entity/Q10929058         literal   
3        uri  http://www.wikidata.org/entity/Q10929058         literal   
4        uri      http://www.wikidata.org/entity/Q5297         literal   
5        uri  http://www.wikidata.org/entity/Q10929058         literal   
6        uri  http://www.wikidata.org/entity/Q10929058         literal   
7        uri  http://www.wikidata.org/entity/Q10929058         literal   

                  ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0  unidad central de procesamiento                  es               uri   
1   modelo de objeto manufacturado                  es               uri   
2   modelo de objeto manufacturado                  es               uri   
3   modelo de 

fals
true
  Clase.type                               Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q62078547         literal   
1        uri   http://www.wikidata.org/entity/Q1188663         literal   
2        uri  http://www.wikidata.org/entity/Q15936437         literal   
3        uri  http://www.wikidata.org/entity/Q23002054         literal   
4        uri   http://www.wikidata.org/entity/Q3354859         literal   
5        uri    http://www.wikidata.org/entity/Q902104         literal   
6        uri   http://www.wikidata.org/entity/Q5009242         literal   

                                   ClaseLabel.value ClaseLabel.xml:lang  \
0              universidad pública de investigación                  es   
1                               Colegios coloniales                  es   
2                      universidad de investigación                  es   
3  institución educativa privada sin fines de lucro                  es   
4                     

true
  Clase.type                              Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q6881511         literal   
1        uri  http://www.wikidata.org/entity/Q4830453         literal   

  ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0          empresa                  es               uri   
1          negocio                  es               uri   

                         claseInicial.value claseInicialLabel.type  \
0  http://www.wikidata.org/entity/Q26215055                literal   
1  http://www.wikidata.org/entity/Q26215055                literal   

  claseInicialLabel.value prop.type                               prop.value  \
0               Q26215055       uri  http://www.wikidata.org/entity/Q1133735   
1               Q26215055       uri  http://www.wikidata.org/entity/Q1133735   

  propLabel.type                propLabel.value propLabel.xml:lang  
0        literal  Westinghouse Electric Company                 es  
1   

true
  Clase.type                        Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q5         literal   

  ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0       ser humano                  es               uri   

                        claseInicial.value claseInicialLabel.type  \
0  http://www.wikidata.org/entity/Q2930888                literal   

  claseInicialLabel.value prop.type                                prop.value  \
0                Q2930888       uri  http://www.wikidata.org/entity/Q18213918   

  propLabel.type propLabel.value propLabel.xml:lang  
0        literal   Alice Recoque                 es  
true
  Clase.type                              Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q4057659         literal   

                     ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0  subdivisión administrativa de Kiev                  es               uri   

                  

fals
true
  Clase.type                              Clase.value ClaseLabel.type  \
0        uri  http://www.wikidata.org/entity/Q1664720         literal   
1        uri   http://www.wikidata.org/entity/Q875538         literal   

      ClaseLabel.value ClaseLabel.xml:lang claseInicial.type  \
0            instituto                  es               uri   
1  universidad pública                  es               uri   

                        claseInicial.value claseInicialLabel.type  \
0  http://www.wikidata.org/entity/Q7003612                literal   
1  http://www.wikidata.org/entity/Q7003612                literal   

  claseInicialLabel.value prop.type                            prop.value  \
0                Q7003612       uri  http://www.wikidata.org/entity/Q4182   
1                Q7003612       uri  http://www.wikidata.org/entity/Q4182   

  propLabel.type                 propLabel.value propLabel.xml:lang  
0        literal  Instituto Weizmann de Ciencias                 es

Es cierto


In [124]:
#Confirmamos que la consulta haya salido bien
clases_object_property_wikidata.head()

,Clase.type,Clase.value,ClaseLabel.type,ClaseLabel.value,ClaseLabel.xml:lang,comic.type,comic.value,comicLabel.type,comicLabel.value,comicLabel.xml:lang,country.type,country.value,countryLabel.type,countryLabel.value,countryLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q859563,literal,Estado laico,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es
1,uri,http://www.wikidata.org/entity/Q99541706,literal,Q99541706,NaN,uri,http://www.wikidata.org/entity/Q10366851,literal,Q10366851,NaN,uri,http://www.wikidata.org/entity/Q30,literal,Estados Unidos,es
2,uri,http://www.wikidata.org/entity/Q1520223,literal,república constitucional,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es
3,uri,http://www.wikidata.org/entity/Q6256,literal,país,es,uri,http://www.wikidata.org/entity/Q11796260,literal,Q11796260,NaN,uri,http://www.wikidata.org/entity/Q36,literal,Polonia,es
4,uri,http://www.wikidata.org/entity/Q842112,literal,Estado socialista,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es


In [125]:
#Aqui se obtienen el numero de apariciones de cada clase para hacer un recuento y obtener la clase a representar
clases_object_property_wikidata["country.value"].value_counts()

http://www.wikidata.org/entity/Q30       7
http://www.wikidata.org/entity/Q148      6
http://www.wikidata.org/entity/Q36       6
http://www.wikidata.org/entity/Q213      2
http://www.wikidata.org/entity/Q15180    2
Name: country.value, dtype: int64

In [ ]:
# Se van comparando cada clase de mayor numero de apariciones a menor numero de apariciones con las clases del value type 
# constraint de la pripiedad de la que se ha realizado la consulta. De esta manera obtendremos aquella clase a representar 
# en el diagrama
